# Batching

In [1]:
import numpy as np

#A class that will do Batching for the algorithm
class Audiobooks_Data_Reader():
    
    def __init__(self, dataset, batch_size = None):
        
        #dataset loads one of "train","Test","Validation"
        npz = np.load('Audiobooks_data_{0}.npz'.format(dataset))
        
        self.inputs, self.targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)
        
        if batch_size is None:
            self.batch_size = self.inputs.shape[0]
        else:
            self.batch_size = batch_size
        self.curr_batch = 0
        self.batch_count = self.inputs.shape[0] // self.batch_size
        
    def __next__(self):
        if self.curr_batch >= self.batch_count:
            self.curr_batch = 0
            raise StopIteration()
            
            
        batch_slice = slice(self.curr_batch*self.batch_size, (self.curr_batch + 1) * self.batch_size)
        inputs_batch = self.inputs[batch_slice]
        targets_batch = self.targets[batch_slice]
        self.curr_batch+=1
        
        
        classes_num = 2
        targets_one_hot = np.zeros((targets_batch.shape[0], classes_num))
        targets_one_hot[range(targets_batch.shape[0]),targets_batch] = 1
        
        return inputs_batch,targets_one_hot
    
    
    
    
    def __iter__(self):
        return self

# Create the Machine Learning Model

In [7]:
import tensorflow as tf

In [12]:
input_size = 10
output_size= 2
hidden_layer_size = 50

tf.reset_default_graph()

inputs = tf.placeholder(tf.float32,[None,input_size])
targets = tf.placeholder(tf.int32, [None,output_size])

weights_1 = tf.get_variable("weights_1",[input_size,hidden_layer_size])
biases_1 = tf.get_variable("biases_1",[hidden_layer_size])

output_1 = tf.nn.relu(tf.matmul(inputs,weights_1)+biases_1)


weights_2 = tf.get_variable("weights_2",[hidden_layer_size,hidden_layer_size])
biases_2 = tf.get_variable("biases_2",[hidden_layer_size])

output_2 = tf.nn.relu(tf.matmul(output_1,weights_2)+biases_2)


weights_3 = tf.get_variable("weights_3",[hidden_layer_size,output_size])
biases_3 = tf.get_variable("biases_3",[output_size])

outputs = tf.matmul(output_2,weights_3)+biases_3


loss = tf.nn.softmax_cross_entropy_with_logits(logits = outputs,labels = targets)
mean_loss = tf.reduce_mean(loss)

optimize = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(mean_loss)

out_equals_target = tf.equal(tf.argmax(outputs,1),tf.argmax(targets,1))
accuracy = tf.reduce_mean(tf.cast(out_equals_target, tf.float32))

sess = tf.InteractiveSession()

initializer = tf.global_variables_initializer()

sess.run(initializer)

batch_size = 100
max_epochs = 50
prev_validation_loss = 999999.0


train_data = Audiobooks_Data_Reader('train',batch_size)
validation_data = Audiobooks_Data_Reader('validation')

for epoch_counter in range(max_epochs):
    
    curr_epoch_loss = 0.
    
    for input_batch,target_batch in train_data:
        _, batch_loss = sess.run([optimize,mean_loss],
                                 feed_dict = {inputs :input_batch,targets :target_batch})
        curr_epoch_loss = curr_epoch_loss + batch_loss
        
    curr_epoch_loss/= train_data.batch_count
    
    validation_loss = 0.
    validation_accuracy = 0.
    
    for input_batch,target_batch in validation_data:
        validation_loss,validation_accuracy = sess.run([mean_loss,accuracy],
                                 feed_dict = {inputs :input_batch,targets: target_batch})
    
    print('Epoch'+str(epoch_counter+1)+
         '.Training Loss: '+'{0:.3f}'.format(curr_epoch_loss)+
         '.Validation Loss: '+'{0:.3f}'.format(validation_loss)+
         '.Validation Accuracy: '+'{0:.2f}'.format(validation_accuracy*100)+'%')
    
    if validation_loss > prev_validation_loss:
        break
    
    prev_validation_loss = validation_loss
    
print('End of Training.')

Epoch1.Training Loss: 0.587.Validation Loss: 0.422.Validation Accuracy: 83.00%
Epoch2.Training Loss: 0.396.Validation Loss: 0.305.Validation Accuracy: 90.38%
Epoch3.Training Loss: 0.330.Validation Loss: 0.263.Validation Accuracy: 91.72%
Epoch4.Training Loss: 0.306.Validation Loss: 0.242.Validation Accuracy: 92.17%
Epoch5.Training Loss: 0.291.Validation Loss: 0.228.Validation Accuracy: 92.84%
Epoch6.Training Loss: 0.280.Validation Loss: 0.219.Validation Accuracy: 93.06%
Epoch7.Training Loss: 0.272.Validation Loss: 0.211.Validation Accuracy: 93.06%
Epoch8.Training Loss: 0.266.Validation Loss: 0.206.Validation Accuracy: 93.06%
Epoch9.Training Loss: 0.261.Validation Loss: 0.202.Validation Accuracy: 93.29%
Epoch10.Training Loss: 0.257.Validation Loss: 0.198.Validation Accuracy: 93.74%
Epoch11.Training Loss: 0.254.Validation Loss: 0.194.Validation Accuracy: 93.96%
Epoch12.Training Loss: 0.252.Validation Loss: 0.192.Validation Accuracy: 93.96%
Epoch13.Training Loss: 0.249.Validation Loss: 0.1

# Test the Model

In [13]:
test_data = Audiobooks_Data_Reader('test')

for input_batch,target_batch in test_data:
        test_accuracy = sess.run([accuracy],
                                 feed_dict = {inputs :input_batch,targets: target_batch})
        
test_accuracy_percent = test_accuracy[0] *100 #Test Accuracy is list

print('Test Accuracy: '+'{0:.2f}'.format(test_accuracy_percent)+'%')
    

Test Accuracy: 88.62%
